# New Token Tracking

This notebook contains commands for setting up and managing token tracking in the Open WebUI system.

## Workflow Overview

For each client, we follow this workflow:

1. **Create a Credit Group**: Create a group for each client and set the token limit for that group
   - The token limit is considered separately for each user in the group
   - Each user gets their own allocation based on the group's limit

2. **Assign Users**: Assign users to the group based on allocated seats under this group/client
   - Users are assigned to groups according to their client's allocated seats
   - Each user within a group has independent token tracking against the group's limit

This allows for per-client token management where each client has their own group with a defined token limit, and users are assigned to groups based on their client's seat allocation.

## Install Upgraded Fork

Install the upgraded fork of the token tracking package.

In [ ]:
!pip install --upgrade --force-reinstall --no-cache-dir git+https://github.com/sarang-remapai/openwebui-token-tracking.git

## First Migration

Initialize the token tracking database with the first migration.

In [ ]:
import os
os.chdir('/Users/sarangali/Workspace/privateai/private_ai_frontend/backend')
os.environ['DATABASE_URL'] = f"sqlite:///{os.getcwd()}/data/webui.db"
!owui-token-tracking init 2>&1

## Token Parity JSON Structure

The `token_parity.json` file defines the token cost structure for different models. It's a JSON array where each object represents a model configuration.

### Example

```json
[
    {
        "provider": "openai",
        "id": "gpt-4.1-mini",
        "name": "GPT-4.1 Mini",
        "input_cost_credits": 1,
        "per_input_tokens": 1,
        "output_cost_credits": 1,
        "per_output_tokens": 1
    }
]

Above settings mean 1 token will be considered as 1 credits, which fits our usecase

```

## Migration Command (Adding New Models)

**Important**: Run this migration command whenever you add new models to the `token_parity.json` file. This command updates the database with the new model configurations.

The migration command reads the `token_parity.json` file and applies any new model definitions to the token tracking system.

In [ ]:
import os
os.chdir('/Users/sarangali/Workspace/privateai/private_ai_frontend/backend')
os.environ['DATABASE_URL'] = "sqlite:////Users/sarangali/Workspace/privateai/private_ai_frontend/backend/data/webui.db"
# Run migration with token_parity.json (located in token-tracking directory)
!owui-token-tracking init --model-json token-tracking/token_parity.json

## Create Plan (Client Group)

Create a credit group for a client with token limits. The token limit is considered separately for each user in the group.

In [1]:
!owui-token-tracking credit-group create "S&L Plan" 1000 "Entry level plan with 1000 token limit"

Successfully created credit group 'S&L Plan'


## Get User ID

Find a user by email address to get their user ID.

In [ ]:
!owui-token-tracking user find --email "sarang@remap.ai"

## Add User to Plan (Client Group)

Add a user to a credit group (client group) using their user ID and the group name. Users are assigned to groups based on allocated seats under the group/client.

In [ ]:
!owui-token-tracking credit-group add-user 39eb28ea-73a2-437a-bc7e-4e0a90529105 "Start Plan"

## Get Usage of User

Check user usage in the interface.